In [1]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

import numpy as np
import matplotlib.pyplot as plt
from scipy import optimize

# 1 - Etude du problème d'optimisation

**Question 1**

Le coût (2) représente les revenus de la boulangerie. En effet, le terme $v^T min \lbrace q, d \rbrace $ représente son chiffre d'affaire, avec $min \lbrace q, d \rbrace $ le vecteur des quantités effectivement vendues de chaque produit (si $d>q$, une partie de la demande n'est pas comblée, tandis que si $d<q$, une partie des produits n'est pas écoulée). Le terme $c^T r$, quant à lui, représente les coûts de production.

**Question 2**

Le terme $min \lbrace q, d \rbrace$ induit une difficulté dans la recherche de maximum car il est non différentiable.

**Question 3**

On pose, pour tout $i\le p$, $h_i(q,d)=\frac {q_i e^{-\alpha q_i}+d_i e^{-\alpha d_i}}{e^{-\alpha q_i}+e^{-\alpha d_i}}$ avec $\alpha$ constante positive. 

On a alors, pour tout q, pour tout d : $h_i(q,d)=\frac {q_i e^{-\alpha (q_i-d_i)}+d_i}{e^{-\alpha (q_i-d_i)}+1}$

Si on fixe q et d et qu'on fait tendre $\alpha$ vers $+\infty$, on a :

Cas 1 : $q_i\ge d_i$, alors $h_i(q,d)$ tend vers $d_i$

Cas 2 : $q_i<d_i$, alors $h_i(q,d)$ tend vers $q_i$

On en déduit que $h_i$ tend, lorsque $\alpha$ tend vers $+\infty$, vers $min(q_i,d_i)$

Pour $\alpha \gg 1$, on en déduit donc que $h$ forme une bonne approximation de la fonction $(q,d)\mapsto (min(q,d))$

La fonction $h$ a par ailleurs la particularité d'être différentiable en tout point de $\mathbb{R}^{2p}$, ce qui nous sera utile dans la recherche d'extrémum.

In [2]:
def h(q, d) :
    return (q*np.exp(-alpha*q) + d*np.exp(-alpha*d)) / (np.exp(-alpha*q) + np.exp(-alpha*d))

**Question 4**

  On s'intéresse donc au problème d'optimisation suivant : 

  $$\min_{\begin{align} Aq-r \leq 0 \\ -q \leq 0 \\ -r \leq 0   \end{align}} (c^Tr-v^Th(q,d))$$

  Où les variables de décisions sont les : q la quantité produite, et r la quantité de matières premières achetée. Cela peut être modélisé par un vecteur de $\R^{m+p}$.

  Les trois contraintes listées représentent respectivement : les quantités minimales de matières premières pour produire les quantités q de produits, le fait de vendre des produits et non d'en acheter, et de même, le fait que l'on ne puisse pas vendre des matières premières. Cette dernière condition est facultative si on fait l'hypothèse tout les coefficients de A son positifs.

# 2 - Etude et résolution numérique

**Question 5**

Nous avons affaire à un problème d'optimisation de $\R^{m+p}$ dans $\R$ avec des contraintes linéaires.

De plus, nous allons considérer que la fonction h est concave composante par composante. Ce n'est rigoureusement pas le cas, mais une vérification numérique de l'allure des courbes confirme cette approximation pour des valeurs élevées de alpha.

In [3]:
def test_h(d = 0, alpha_ = 1) :
    global alpha
    alpha = alpha_
    
    x = np.linspace(d-5, d+5, 100)
    y = h(x, d)
    min_y = [min(varx, d) for varx in x]
    plt.plot(x, y, label="h(x, d)")
    plt.plot(x, min_y, label="min(x, d)")
    plt.xlabel("x")
    plt.title("comparaison de h(d, x) et min(d, x)")
    plt.legend()

interact(test_h, d=(-10, 10, 0.1), alpha=(0.5, 10, 0.1));

interactive(children=(FloatSlider(value=0.0, description='d', max=10.0, min=-10.0), IntSlider(value=1, descrip…

La fonction à minimiser est donc une fonction convexe en tant que somme d'une fonction affine et d'une fonction convexe. Cela nous permet d'envisager des méthodes de recherche de point selle somme les algorithmes d'Uzawa ou d'Arrow-Hurwicz.

**Question 6**

In [4]:
m = 5
p = 3

alpha = 0.1
c = 1e-3 * np.array([30, 1, 1.3, 4, 1])
v = np.array([0.9, 1.5, 1.1])
d = np.array([400, 67, 33])
A = np.array([[3.5, 2,   1 ],
              [250, 80,  25],
              [0,   8,   3 ],
              [0,   40,  10],
              [0,   8.5, 0 ]])

In [5]:
def fun(grand_vecteur) :
    q = grand_vecteur[:p]
    r = grand_vecteur[p:]
    return np.dot(c, r) - np.dot(v, h(q, d))

def lagrangien(grand_vecteur, lambda_):
    #print(str(fun(grand_vecteur) - np.dot(lambda_, contraintes(grand_vecteur))) + ", " + str(fun(grand_vecteur)) + ", " + str(np.dot(lambda_, contraintes(grand_vecteur))))
    return fun(grand_vecteur) - np.dot(lambda_, contraintes(grand_vecteur))

def contraintes(grand_vecteur):
    q = grand_vecteur[:p]
    r = grand_vecteur[p:]
    
    const = np.zeros(2*m+p)
    const[:m] = r - np.dot(A, q)
    const[m:2*m] = r
    const[-p:] = q

    # Les contraintes sont ici inversées car les algorithmes d'optimisation de scipy utilisent la convention ci(x) >= 0
    return const

In [6]:
def uzawa(x0, lambda_0, alpha_uzawa, f, c):
    x = x0
    l = lambda_0
    prev_x = x0 + np.ones(len(x0))
    mybounds = [(-100, 2000) for elem in x0]
    # Les contraintes servent ici à guider le solveur SLSQP pour qu'il n'y ait pas d'évaluation de h avec des valeurs trop élevées,
    # mais elles ne doivent pas interférer avec la véritable résolution.
    
    while np.linalg.norm(x - prev_x) > 0.1 :
        prev_x = x
        x = (optimize.minimize(f, x, args=l, method='SLSQP', bounds = mybounds)).x
        l = np.maximum(l - alpha_uzawa*c(x), 0)
    return x,l

In [7]:
x0 = np.zeros(m+p)
lambda_0 = np.ones(2*m+p)
alpha_uzawa = 0.1
alpha = 0.1

x, l = uzawa(x0, lambda_0, alpha_uzawa, lagrangien, contraintes)
print("q : " + str(x[:p]))
print("r : " + str(x[p:]))

q : [-100. -100. -100.]
r : [2000. 2000. 2000. 2000. 2000.]


Malheureusement, on se rend compte que le résultat n'est pas satisfaisant, et que le solveur ne donne pas des quantités acceptables. Une des raisons de cet échec peut être l'initialisation des variables comme x0 ou lambda_0, ou le fait que notre fonction h n'est pas exactement convexe.


Nous allons plutôt utiliser les méthodes offertes par la librairie scipy.optimize :

In [8]:
def f(alpha_interact) :
    global alpha
    alpha = alpha_interact
    x0 = np.zeros(m+p)
    x = optimize.minimize(fun, x0, method='SLSQP', constraints={'type':'ineq', 'fun':contraintes}, options={'maxiter':1000, 'disp':True}).x
    print("q : " + str(x[:p]))
    print("r : " + str(x[p:]))

interact(f, alpha_interact = (0.05, 2, 0.05));

interactive(children=(FloatSlider(value=1.0, description='alpha_interact', max=2.0, min=0.05, step=0.05), Outp…

Cette fois-ci, l'agorithme nous donne des valeurs plus judicieuses, et qui correspondent à l'intuition (à savoir vendre autant que la demande et acheter les quantités en consequences)

**Question 7**

***a)***

On cherche ici à maximiser l'espérance du profit : $\sum_{k=1}^K \pi_k (v^T h(q, d_k) - c^T r)$
Le problème devient donc le suivant : 

  $$\min_{\begin{align} Aq-r \leq 0 \\ -q \leq 0 \\ -r \leq 0   \end{align}} \sum_{k=1}^K \pi_k (c^T r - v^T h(q, d_k))$$


In [9]:
d1 = np.array([400, 67, 33])
d2 = np.array([500, 80, 53])
d3 = np.array([300, 60, 43])

proba = [(d1, 0.5), (d2, 0.3), (d3, 0.2)]

In [10]:
def fun_proba(grand_vecteur) :
    q = grand_vecteur[:p]
    r = grand_vecteur[p:]
    scenarii = [p * (np.dot(c, r) - np.dot(v, h(q, d))) for (d, p) in proba]
    return np.sum(scenarii)

def lagrangien_proba(grand_vecteur, lambda_):
    #print(str(fun(grand_vecteur) - np.dot(lambda_, contraintes(grand_vecteur))) + ", " + str(fun(grand_vecteur)) + ", " + str(np.dot(lambda_, contraintes(grand_vecteur))))
    return fun_proba(grand_vecteur) - np.dot(lambda_, contraintes(grand_vecteur))

In [11]:
def f2(alpha_interact) :
    global alpha
    alpha = alpha_interact
    x0 = np.zeros(m+p)
    x = optimize.minimize(fun_proba, x0, method='SLSQP', constraints={'type':'ineq', 'fun':contraintes}, options={'maxiter':1000, 'disp':True}).x
    print("q : " + str(x[:p]))
    print("r : " + str(x[p:]))

interact(f2, alpha_interact = (0.05, 2, 0.05));

interactive(children=(FloatSlider(value=1.0, description='alpha_interact', max=2.0, min=0.05, step=0.05), Outp…

On remarque que les résultats ont l'air assez stables en fonction de alpha, et semblent se rapprocher des valeurs q=[400, 80, 53]. On remarque aussi qu'ils ne semblent pas résulter de la moyenne des différentes demandes.

# 3 - Etude du problème non-régularisé

**Question 8**

***a)***
Le problème d'optimisation est le suivant :
  $$\min_{\begin{align} Aq-r \leq 0 \\ -q \leq 0 \\ -r \leq 0   \end{align}} \sum_{k=1}^K \pi_k (c^T r - v^T min(q, d_k)))$$

***b)***
Mq Aq = r à l'optimum

Supposons que $\exists i_0, \; (Aq)_{i_0} < r_{i_0}$

Notre problème est un problème d'optimisation convexe à contraintes affines d'ensemble admissible non vide, donc il existe une solution à ce problème. Notons $(q^*, r^*)$ les quantités à l'optimum.

Considérons $(\hat{r}, q^*)$ tel que $$\hat{r}_i = r^*_i \;si\; i \neq i_0,  \\  \hat{r}_i = \frac{r^*_i + (Aq)_i}{2} \;si\; i = i_0$$
On vérifie aisément que $(\hat{r}, q^*)$ vérifie les contraintes, et on a également $c^T\hat{r} < c^T r^*$, donc le profit total est plus élevé. Cela implique que $f(\hat{r}, q^*) < f(r^*, q^*)$ (en notant f la fonction à minimiser de notre problème tel que défini en a). ). On aboutit à une contradiction car $\forall (q, r) \in \R^p\times\R^m\; f(r, q) < f(r^*, q^*)$ par définition de $(r^*, q^*)$.

On en déduit que la contrainte (1) doit être active.

**c)** À l'équilibre, r est donc entièrement déterminé par q avec la relation $Aq=r$.

On peut donc réécrire le problème d'optimisation sous la forme :
$$\min_{\begin{align} -q \leq 0 \end{align}} \sum_{k=1}^K \pi_k (c^T Aq - v^T min(q, d_k))$$

La condition $-r\le 0$, c'est-à-dire $-Aq\le 0$ est impliquée par la condition $-q\le 0$.

De plus, on observe que $c^T A-v^T<0$ et $d\ge 0$, ce qui fait que lrosqu'on prend un coefficient de $q$ négatif, la situation est moins optimale que lorsqu'on remplace ce coefficient négatif par un zéro. La condition $-q\le 0$ n'a donc pas d'influence sur la recherche de minimum ici. Le problème se réécrit donc :
$$\min_{\begin{align} q \end{align}} \sum_{k=1}^K \pi_k (c^T Aq - v^T min(q, d_k))$$

On obtient un problème d'optimisation sans contrainte

**Question 9**

**a)** On a :

$$\max_{\begin{align} u_k\le d_k, u_k\le q \end{align}}(v^T u_k)=\max_{\begin{align} u_k\le \min\lbrace d,q\rbrace \end{align}}(v^T u_k)$$

Or $v\ge 0$ donc lorsqu'on maximise $v^T u_k$, la contrainte inégalité sera active, ce qui devient : $\max{v^T \min\lbrace d,q \rbrace}$.

Il y a donc équivalence entre ces deux problèmes d'optimisation.

**b)** Le problème d'optimisation de la question 8 devient alors :

$$\min_{\begin{align} q,u_k\\ u_k\le d_k \\ u_k\le q \end{align}} \sum_{k=1}^K \pi_k (c^T Aq - v^T u_k)$$

Cette nouvelle formulation présente l'avantage d'être différentiable.

**c)**

In [12]:
d1=np.array([400,67,33])
d2=np.array([500,80,53])
d3=np.array([300,60,43])
pi=np.array([0.5,0.3,0.2])

def contraintes_2(UQ):
    u=UQ[0:9]
    q=UQ[9:12]
    C=np.zeros(18)
    for i in range(3):
        C[i]=d1[i]-u[i]
        C[i+3]=q[i]-u[i]
        C[i+6]=d2[i]-u[i+3]
        C[i+9]=q[i]-u[i+3]
        C[i+12]=d3[i]-u[i+6]
        C[i+15]=q[i]-u[i+6]
    return C
def fonction(UQ):
    u=UQ[0:9]
    q=UQ[9:12]
    s=0
    for i in range(3):
        uk=u[i:i+3]
        s+=pi[i]*(np.dot(np.dot(np.transpose(c),A),q)-np.dot(np.transpose(v),uk))
    return s

x0 = np.zeros(12)
x = optimize.minimize(fonction, x0, method='SLSQP', constraints={'type':'ineq', 'fun':contraintes_2}, options={'maxiter':1000, 'disp':True}).x
print("u=",x[:9], "et q=",x[9:12])
print("Bénéfice :",-fonction(x),"euros")

Optimization terminated successfully    (Exit mode 0)
            Current function value: -414.89000000204305
            Iterations: 32
            Function evaluations: 416
            Gradient evaluations: 32
u= [400.  67.  33. 500.  67.   0.   0.   0.   0.] et q= [500.  67.  33.]
Bénéfice : 414.89000000204305 euros


On remarque que l'on a un bénéfice plus important (414€ contre 319€ précédemment) en regardant la valeur exacte de la fonction, et pas une valeur approchée.

**Question 10**

Pour résoudre ce problème de manière non lisse, on peut utiliser un algorithme du point proximal. Implémentons-le ci-dessous :

In [37]:
D=np.array([d1,d2,d3])
def fonction2(q):
    s=0
    for i in range(3):
        mini=np.zeros(3)
        for j in range(3):
            mini[j]=min(q[j],D[i][j])
        s+=pi[i]*(np.dot(np.dot(np.transpose(c),A),q)-np.dot(np.transpose(v),mini))
    return s

def prox(q,l):
    derivee=np.zeros(3)
    for i in range(3):
        mini=q<D[i]
        derivee+=pi[i]*(np.dot(np.transpose(c),A)-np.dot(np.transpose(v),mini))
    s=q-l*derivee
    return s

def grad_proximal():
    q=np.array([0,0,0])
    l=100
    for n in range(1,10000):
        q=prox(q,l)
        l=100/n
    return q,fonction2(q)
grad_proximal()

(array([400.00275653, 438.94529228, 676.26822486]),
 np.float64(-142.48718454813223))